<a href="https://colab.research.google.com/github/Wadaken43/DeepLearning/blob/main/darknet_53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from torchinfo import summary
import torch
import torch.nn as nn
import torch.nn.functional as F


DarkNet-53のブロックの定義

In [4]:
class DarknetBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarknetBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, in_channels // 2, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels // 2)
        self.conv2 = nn.Conv2d(in_channels // 2, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(in_channels)

#ResNetの残差ブロックと似ている。
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.leaky_relu(out, inplace=True)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.leaky_relu(out, inplace=True)
        out += residual
        return out


In [5]:
class Darknet53(nn.Module):
    def __init__(self, num_classes=1000):
        super(Darknet53, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = nn.Sequential(
            DarknetBlock(32),
            DarknetBlock(64)
        )

        self.layer2 = nn.Sequential(
            DarknetBlock(64),
            DarknetBlock(128),
            DarknetBlock(128),
            DarknetBlock(64)
        )

        self.layer3 = nn.Sequential(
            DarknetBlock(64),
            DarknetBlock(128),
            DarknetBlock(128),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(256),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(256)
        )

        self.layer4 = nn.Sequential(
            DarknetBlock(256),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(512),
            DarknetBlock(1024),
            DarknetBlock(1024),
            DarknetBlock(1024),
            DarknetBlock(512)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
       out = self.conv1(x)
       out = self.bn1(out)
       out = F.leaky_relu(out, inplace=True)
       out = F.max_pool2d(out, kernel_size=2, stride=2)

       out = self.layer1(out)
       out = F.max_pool2d(out, kernel_size=2, stride=2)

       out = self.layer2(out)
       out = F.max_pool2d(out, kernel_size=2, stride=2)

       out = self.layer3(out)
       out = F.max_pool2d(out, kernel_size=2, stride=2)

       out = self.layer4(out)

       out = self.avgpool(out)
       out = torch.flatten(out, 1)
       out = self.fc(out)

       return out


モデルの可視化

In [9]:
model=Darknet53()
summary(model) #本来は入力サイズも渡す。


Layer (type:depth-idx)                   Param #
Darknet53                                --
├─Conv2d: 1-1                            864
├─BatchNorm2d: 1-2                       64
├─Sequential: 1-3                        --
│    └─DarknetBlock: 2-1                 --
│    │    └─Conv2d: 3-1                  512
│    │    └─BatchNorm2d: 3-2             32
│    │    └─Conv2d: 3-3                  4,608
│    │    └─BatchNorm2d: 3-4             64
│    └─DarknetBlock: 2-2                 --
│    │    └─Conv2d: 3-5                  2,048
│    │    └─BatchNorm2d: 3-6             64
│    │    └─Conv2d: 3-7                  18,432
│    │    └─BatchNorm2d: 3-8             128
├─Sequential: 1-4                        --
│    └─DarknetBlock: 2-3                 --
│    │    └─Conv2d: 3-9                  2,048
│    │    └─BatchNorm2d: 3-10            64
│    │    └─Conv2d: 3-11                 18,432
│    │    └─BatchNorm2d: 3-12            128
│    └─DarknetBlock: 2-4                 --
│    │